# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
J. Li  ->  J. Li  |  ['J. Li']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
J. Liu  ->  J. Liu  |  ['J. Liu']
J. Li  ->  J. Li  |  ['J. Li']


J. Li  ->  J. Li  |  ['J. Li']
T. Henning  ->  T. Henning  |  ['T. Henning']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']
K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']
A. Somigliana  ->  A. Somigliana  |  ['A. Somigliana']
Arxiv has 80 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2508.00978


extracting tarball to tmp_2508.00978...

 done.
  0: tmp_2508.00978/aassymbols.tex, 579 lines
  1: tmp_2508.00978/ms.tex, 1,191 lines


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 2 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2508.00978/ms.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)


...

extracting tarball to tmp_2508.02162... done.
Retrieving document from  https://arxiv.org/e-print/2508.02576


extracting tarball to tmp_2508.02576...

 done.


T. Henning  ->  T. Henning  |  ['T. Henning']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']
K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']
A. Somigliana  ->  A. Somigliana  |  ['A. Somigliana']


Found 155 bibliographic references in tmp_2508.02576/main.bbl.
Issues with the citations
syntax error in line 88: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.00978-b31b1b.svg)](https://arxiv.org/abs/2508.00978) | **Mapping the Distant and Metal-Poor Milky Way with SDSS-V**  |
|| V. Chandra, et al. -- incl., <mark>H.-W. Rix</mark> |
|*Appeared on*| *2025-08-05*|
|*Comments*| *31 pages, 19 figures; Submitted to AAS Journals;*|
|**Abstract**|            The fifth-generation Sloan Digital Sky Survey (SDSS-V) is conducting the first all-sky low-resolution spectroscopic survey of the Milky Way's stellar halo. We describe the stellar parameter pipeline for the SDSS-V halo survey, which simultaneously models spectra, broadband photometry, and parallaxes to derive stellar parameters, metallicities, alpha abundances, and distances. The resulting BOSS-MINESweeper catalog is validated across a wide range of stellar parameters and metallicities using star clusters and a comparison to high-resolution spectroscopic surveys. We demonstrate several scientific capabilities of this dataset: identifying the most chemically peculiar stars in our Galaxy, discovering and mapping distant halo substructures, and measuring the all--sky dynamics of the Milky Way on the largest scales. The BOSS-MINESweeper catalog for SDSS DR19 is publicly available and will be updated for future data releases.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.02576-b31b1b.svg)](https://arxiv.org/abs/2508.02576) | **MINDS. Young binary systems with JWST/MIRI: Variable water-rich primaries and extended emission**  |
|| N. T. Kurtovic, et al. -- incl., <mark>T. Henning</mark>, <mark>G. Perotti</mark>, <mark>K. Schwarz</mark>, <mark>A. Somigliana</mark> |
|*Appeared on*| *2025-08-05*|
|*Comments*| *Accepted for publication in A&A, 26 pages, 20 Figures. Shortened abstract in arxiv*|
|**Abstract**|            As part of the JWST GTO program MINDS, we analyze the mid-infrared emission of three Class II binary systems: VW Cha, WX Cha, and RW Aur, to investigate the impact of stellar multiplicity on the chemistry and physics of their inner disk. We analyze the 1D spectrum from JWST/MIRI-MRS for primary and secondary disks separately, extracted by combining forward modeling with a theoretical PSF and aperture photometry. We modeled the molecular lines with 0D slab models. We interpret the results by comparing our JWST spectra to VLT/CRIRES+, Spitzer/IRS, and ALMA. Primary and secondary disks are dramatically different in their mid-infrared emission, with primary disks showing H2O-rich spectra, and secondary disks being mostly line poor to the sensitivity of our spectra. When comparing MIRI-MRS to Spitzer/IRS, we observe large variability in the line emission of VW Cha A, as well as in the continuum of RW Aur A. The disks around VW Cha BC and RW Aur B show evidence of ionizing radiation, and a further comparison with ALMA at high angular resolution dust continuum suggest that the spectrum of RW Aur B is well explained by its ~4 au cavity. All the systems show [Ne II] jet emission, and three of them also show spatially resolved emission structures in H2, likely originated by outflows and dynamical interactions. Many of the observed features in the primary disks, such as enhanced water emission, could be linked to the increased accretion and radial drift produced by dynamical disk truncation. However, additional mechanisms are needed to explain the large differences between primary and secondary disks, potentially inner disk substructures. This work is an example of the need for combining multiple facilities to fully understand the observations from JWST.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.01413-b31b1b.svg)](https://arxiv.org/abs/2508.01413) | **Measurement of the Primary Beam of the Tianlai Cylindrical Antenna Using an Unmanned Aerial Vehicle**  |
|| <mark>J. Li</mark>, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-08-05*|
|*Comments*| *19 pages, 16 figures, RAA accepted*|
|**Abstract**|            The Tianlai Cylinder Pathfinder Array consists of three adjacent cylindrical reflectors fixed on the ground, each 40 meters long and 15 meters wide, with the cylinder axis oriented along the North-South (N-S)direction. Dual linear polarisation feeds are distributed along the focus line, parallel to the cylinder axis. Measurement of the primary beam profile of these cylindrical reflectors is difficult, as they are too large to be placed in an anechoic chamber. While the beam profile along the East-West (E-W) direction can be measured with the transit observations of bright astronomical radio sources, the beam profile along the N-S direction remains very uncertain. We present a preliminary measurement of the beam profile of the Tianlai cylindrical antenna along both the N-S direction and E-W direction in the frequency range of 700-800 MHz, using a calibrator source carried by an unmanned aerial vehicle (UAV) flying in the far field. The beam profile of the Tianlai cylindrical antenna is determined from the analysis of the auto-correlation signals from the the cylinder array correlator, taking into account the emitter antenna beam profile, itself measured with a dipole antenna on the ground. The accuracy of the UAV-based determination of the cylinder beam profiles is validated by comparing the results with the one derived from bright astronomical source transits, and with simulated beams.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.01428-b31b1b.svg)](https://arxiv.org/abs/2508.01428) | **SN 2024gy: Multi-epoch Spectroscopic Evidence for Delayed Detonation in a Type Ia Supernova**  |
|| L. Li, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2025-08-05*|
|*Comments*| *21 pages, 11 figures, submitted to ApJ*|
|**Abstract**|            We present photometric and spectroscopic observations of SN 2024gy, a Type Ia supernova (SN Ia) exhibiting high-velocity features (HVFs) in its early-time spectra. This SN reaches a peak $B$-band magnitude of $-19.25 \pm 0.28$ mag and subsequently declines by $\Delta m_{15}(B) \approx 1.12$ mag, consistent with the luminosity-width relation characteristic of normal SNe Ia. Based on the peak thermal luminosity of $(1.2 \pm 0.3) \times 10^{43}$ erg s$^{-1}$, we estimate that $0.57 \pm 0.14~\rm M_{\odot}$ of $^{56}$Ni was synthesized during the explosion. Our dense early spectral monitoring revealed significant velocity disparities within the ejecta. Notably, absorption features from the \CaII\ near-infrared triplet were observed at velocities exceeding 25,000 km s$^{-1}$, while the \SiII\, \ld 6355 line velocity at the same epoch was significantly lower at $\sim$ 16,000 km s$^{-1}$. This velocity disparity likely reflects distinct ionization states of intermediate-mass elements in the outermost layers. The prominent \CaII\, HVFs may originate from ionization suppression within the highest-velocity ejecta, potentially indicative of minimal hydrogen mixing in a delayed-detonation explosion scenario. Additionally, the Ni/Fe ratio derived from the nebular spectrum of SN 2024gy provides further support for this model.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.01596-b31b1b.svg)](https://arxiv.org/abs/2508.01596) | **Identifying Radio Active Galactic Nuclei with Machine Learning and Large-Area Surveys**  |
|| X.-L. Fan, <mark>J. Li</mark> |
|*Appeared on*| *2025-08-05*|
|*Comments*| *10 pages, 6 figures, accepted for publication in A&A*|
|**Abstract**|            Context. Active galactic nuclei (AGNs) and star forming galaxies (SFGs) are the primary sources of extragalactic radio sky. But it is difficult to distinguish the radio emission produced by AGNs from that by SFGs, especially when the radio sources are faint. Best et al. (2023) classified the radio sources in LoTSS Deep Fields DR1 through multiwavelength SED fitting. With the classification results of them, we perform a supervised machine learning to distinguish radio AGNs and radio SFGs. Aims. We aim to provide a supervised classifier to identify radio AGNs, which can get both high purity and completeness simultaneously, and can easily be applied to datasets of large-area surveys. Methods. The classifications of Best et al. (2023) are used as the true labels for supervised machine learning. With the cross-matched sample of LoTSS Deep Fields DR1, AllWISE and Gaia DR3, the features of optical and mid-infrared magnitude and colors, are applied to train the classifier. The performance of the classifier is evaluated mainly by the precission, recall and F1 score of both AGNs and non-AGNs. Results. By comparing the performance of six learning algorithms, CatBoost is chosen to construct the best classifier. The best classifier get precision = 0.974, recall = 0.865 and F1 = 0.916 for AGNs, precision = 0.936, recall = 0.988 and F1 = 0.961 for non-AGNs. After applying our classifier to the cross-matched sample of LoTSS DR2, AllWISE and Gaia DR3, we obtain a sample of 49716 AGNs and 102261 non-AGNs. The reliability of these classification results is confirmed by comparing with the spectroscopic classification of SDSS. The precission and recall of AGN sample can be as high as 94.2% and 92.3%, respectively. We also train a model to identify radio excess sources. The F1 scores are 0.610 and 0.965 for sources with and without radio excess, respectively.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.02162-b31b1b.svg)](https://arxiv.org/abs/2508.02162) | **Discovery and dynamics of a Sedna-like object with a perihelion of 66 au**  |
|| Y.-T. Chen, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-08-05*|
|*Comments*| *Accepted manuscript of an article published open access in Nature Astronomy under a Creative Commons Attribution-NonCommercial-NoDerivatives (CC BY-NC-ND 4.0) license. The final published version is available at this https URL*|
|**Abstract**|            Trans-Neptunian objects (TNOs) with large perihelion distances ($q > 60$ au) and semi-major axes ($a > 200$ au) provide insights into the early evolution of the solar system and the existence of a hypothetical distant planet. These objects are still rare and their detection is challenging, yet they play a crucial role in constraining models of solar system formation. Here we report the discovery of a Sedna-like TNO, 2023\,KQ$_{14}$, nicknamed `Ammonite', with $q = 66$ au, $a = 252$ au, and inclination $i=11^\circ$. Ammonite's orbit does not align with those of the other Sedna-like objects and fills the previously unexplained `$q$-gap' in the observed distribution of distant solar system objects. Simulations demonstrate that Ammonite is dynamically stable over 4.5 billion years. % with less than 1\% variation in its semi-major axis. Our analysis suggests that Ammonite and the other Sedna-like objects may have shared a primordial orbital clustering around 4.2 billion years ago. Furthermore, Ammonite's stable orbit favors larger orbits ($\sim$ 500 au) rather than closer ones for a large hypothetical planet in present-day trans-Neptunian space.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2508.00978/./sgrsel.png', 'tmp_2508.00978/./sgr_vgsr.png', 'tmp_2508.00978/./feh_comparison.png', 'tmp_2508.00978/./afe_comparison.png', 'tmp_2508.00978/./kgsel.png']
copying  tmp_2508.00978/./sgrsel.png to _build/html/
copying  tmp_2508.00978/./sgr_vgsr.png to _build/html/
copying  tmp_2508.00978/./feh_comparison.png to _build/html/
copying  tmp_2508.00978/./afe_comparison.png to _build/html/
copying  tmp_2508.00978/./kgsel.png to _build/html/
exported in  _build/html/2508.00978.md
    + _build/html/tmp_2508.00978/./sgrsel.png
    + _build/html/tmp_2508.00978/./sgr_vgsr.png
    + _build/html/tmp_2508.00978/./feh_comparison.png
    + _build/html/tmp_2508.00978/./afe_comparison.png
    + _build/html/tmp_2508.00978/./kgsel.png
found figures ['tmp_2508.02576/./figures/VWCha/VWCha_h2o_comp_v2.png', 'tmp_2508.02576/./figures/RWAur/RWAurB_manual_fit.png', 'tmp_2508.02576/./figures/spec_all_jwstmiri_scaled.png']
copying  tmp_2508.02576/./figures/VWCha/VWCha_h2o_comp_v2.png

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\@}{outputaffil@separated $
$}$</div>



<div id="title">

# Mapping the Distant and Metal-Poor Milky Way with SDSS-V

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2508.00978-b31b1b.svg)](https://arxiv.org/abs/2508.00978)<mark>Appeared on: 2025-08-05</mark> -  _31 pages, 19 figures; Submitted to AAS Journals;_

</div>
<div id="authors">

V. Chandra, et al. -- incl., <mark>H.-W. Rix</mark>

</div>
<div id="abstract">

**Abstract:** $\noindent$ The fifth-generation Sloan Digital Sky Survey (SDSS-V) is conducting the first all--sky low--resolution spectroscopic survey of the Milky Way's stellar halo.We describe the stellar parameter pipeline for the SDSS-V halo survey, which simultaneously models spectra, broadband photometry, and parallaxes to derive stellar parameters, metallicities, alpha abundances, and distances.The resulting $\minesweepercat$ catalog is validated across a wide range of stellar parameters and metallicities using star clusters and a comparison to high-resolution spectroscopic surveys.We demonstrate several scientific capabilities of this dataset: identifying the most chemically peculiar stars in our Galaxy, discovering and mapping distant halo substructures, and measuring the all--sky dynamics of the Milky Way on the largest scales.The $\minesweepercat$ catalog for SDSS DR19 is publicly available and will be updated for future data releases.

</div>

<div id="div_fig1">

<img src="tmp_2508.00978/./sgrsel.png" alt="Fig9.1" width="50%"/><img src="tmp_2508.00978/./sgr_vgsr.png" alt="Fig9.2" width="50%"/>

**Figure 9. -** The Sagittarius Stream (Sgr) in SDSS-V.
    **Top:** Selection of Sgr members in angular momentum space. Stars lying very close to the orbital plane of Sgr are used to define a polygon that captures most likely Sgr members.
    **Bottom:** The Galactocentric radial velocity of stars beyond 25 kpc as a function of longitude along the Sgr stream, with angular momentum-selected Sgr stars highlighted in blue.
    Our selection is effective at excising Sgr stars from the field halo.  (*fig:sgr*)

</div>
<div id="div_fig2">

<img src="tmp_2508.00978/./feh_comparison.png" alt="Fig2.1" width="50%"/><img src="tmp_2508.00978/./afe_comparison.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** Mean metallicity and alpha abundance for calibration clusters in SDSS-V, as compared to parameters from the literature.
    The underlying data and references are listed in Table \ref{tab:clusters}.
    The 1-1 line is overlaid, along with the constant offset measured as the median difference between \minesweeper  and literature.
     (*fig:cluster_means*)

</div>
<div id="div_fig3">

<img src="tmp_2508.00978/./kgsel.png" alt="Fig6" width="100%"/>

**Figure 6. -** Performance of the distant halo selection carton.
    Blue (red) points show giants targeted to lie beyond $d_\mathrm{helio} > 10 (30)$ kpc.
    The top panel shows the marginal distribution of $r_\mathrm{gal}$.
    Gold points show a cross-match to the dwarf spheroidal galaxy member catalog from \cite{Walker2023}.
    Our parallax and color-based target selection is highly efficient at identifying the most distant stars in the MW halo. (*fig:kg*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2508.00978"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\pageref}{\oldpageref*}$</div>



<div id="title">

# MINDS. Young binary systems with JWST/MIRI: \\variable water-rich primaries and extended emission

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2508.02576-b31b1b.svg)](https://arxiv.org/abs/2508.02576)<mark>Appeared on: 2025-08-05</mark> -  _Accepted for publication in A&A, 26 pages, 20 Figures. Shortened abstract in arxiv_

</div>
<div id="authors">

N. T. Kurtovic, et al. -- incl., <mark>T. Henning</mark>, <mark>G. Perotti</mark>, <mark>K. Schwarz</mark>, <mark>A. Somigliana</mark>

</div>
<div id="abstract">

**Abstract:** Dynamical disk-companion interactions can have a large impact on the evolution of circumstellar disks, as these can produce perturbations to the material distribution, density, and temperature, affecting their potential for planet formation. As part of the JWST GTO program MINDS, we analyze the mid-infrared emission of three Class II binary systems: VW Cha, WX Cha, and RW Aur, to investigate the impact of stellar multiplicity on the chemistry and physics of their inner disk. We analyze the 1D spectrum from JWST/MIRI-MRS for primary and secondary disks separately, extracted by combining forward modeling with a theoretical PSF and aperture photometry. After continuum subtraction, we modeled the molecular lines with 0D slab models. We interpret the results by comparing our JWST spectra to VLT/CRIRES+, Spitzer/IRS. The extended mid-infrared emission is compared to ALMA, for which we also include the binary DF Tau in our sample. Primary and secondary disks are dramatically different in their mid-infrared emission, with primary disks showing $H_2$ O-rich spectra, and secondary disks being mostly line poor to the sensitivity of our spectra. When comparing MIRI-MRS to Spitzer/IRS, we observe large variability in the line emission of VW Cha A, as well as in the continuum of RW Aur A. The disks around VW Cha BC and RW Aur B show evidence of ionizing radiation, and a further comparison with ALMA at high angular resolution dust continuum suggest that the spectrum of RW Aur B is well explained by its $\sim 4$ au cavity.    All the systems show [ Ne II ] jet emission, and three of them also show spatially resolved emission structures in $H_2$ , likely originated by outflows and dynamical interactions. Many of the observed features in the primary disks, such as enhanced water emission, could be linked to the increased accretion and radial drift produced by dynamical disk truncation. However, additional mechanisms are needed to explain the large differences between primary and secondary disks, potentially inner disk substructures. This work is an example of the need for combining multiple facilities to fully understand the observations from JWST.

</div>

<div id="div_fig1">

<img src="tmp_2508.02576/./figures/VWCha/VWCha_h2o_comp_v2.png" alt="Fig4" width="100%"/>

**Figure 4. -** Comparison of the JWST/MIRI-MRS spectra and Spitzer/IRS in mode SH and LH. The panels **(a)**, **(b)**, and **(c)** show the comparison of MIRI-MRS with native resolution in grey, and in red after being convolved to the IRS resolution. The IRS spectra is shown in blue. The residuals from subtracting the IRS spectra to the MIRI-MRS are shown in green. Panel **(d)** is a zoom into the [Ne II] emission, with two vertical dashed lines showing the location of the [Ne II] at rest and the wavelength of [Ne II] with -180 km s$^{-1}$ of blueshift. The emission of VW Cha BC is also shown, and convolved to the resolution of the IRS. Panel **(e)** is a zoom into the region with the main emission of HCN and $CO_2$. For comparison, the residuals from subtracting the IRS observation to the MIRI-MRS observation are shown both with their real flux, and with the MIRI-MRS flux scaled by 0.5. Panel **(f)** shows the region with the most prominent cold $H_2$O lines. Even after scaling MIRI-MRS by 0.5, there are still residuals consistent with a 200 K slab. (*fig:vwcha:comp-spt-jwst*)

</div>
<div id="div_fig2">

<img src="tmp_2508.02576/./figures/RWAur/RWAurB_manual_fit.png" alt="Fig15" width="100%"/>

**Figure 15. -**  The molecular line emission of RW Aur B. In the top is the original continuum subtracted spectra, with the same continuum subtraction algorithm used for the primary disks. In the middle is the spectra after filtering the signals with frequency smaller than than 0.5 px$^{-1}$. In the bottom, the spectra after frequency filtering and subtracting a masked average with a window size of 10 px. For reference, slab models for HCN, $H_2$O, and $CO_2$, are shown with the original and with the modified spectra.  (*fig:app:rwaurb_manual_fit*)

</div>
<div id="div_fig3">

<img src="tmp_2508.02576/./figures/spec_all_jwstmiri_scaled.png" alt="Fig2" width="100%"/>

**Figure 2. -** JWST/MIRI-MRS spectra for the three binary systems in our sample, scaled to 150 pc for comparison. The spectra of the secondary sources are enlarged in the insert, showing the observed flux between the MIRI-MRS band 1A and 3C.  (*fig:all_jwstmiri*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2508.02576"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

134  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
